In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=200,
    return_messages=True,
)

examples = [
    {
        "question": "인터스텔라",
        "answer": "🚀👩‍🚀🪐",
    }
]

example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{question}"),
    ("ai", "{answer}")
])

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "너는 사람에게 도움을 주는 유능한 AI 챗봇이야. 질문이 영화 이름인 경우에만 example과 같은 방식으로 알려줘."),
        MessagesPlaceholder(variable_name="history"),
        example_prompt,
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

In [2]:
invoke_chain("탑건")

content='✈️🔥🎖️'


In [3]:
invoke_chain("조커")

content='🃏🎭🔪'


In [4]:
invoke_chain("내가 처음에 물어본 영화가 뭐였지?")

content='처음에 물어본 영화는 "탑건"이었습니다.'
